In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import glob
import os
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from numpy import linalg

In [2]:
def mean(Xtrain):
    return np.mean(Xtrain,axis = 1)
#     mean = [0] * Xtrain.shape[0]
#     mean = np.array(mean)
#     for col in Xtrain.T:
#         mean = mean + col
#     N = Xtrain.shape[1]
#     mean = mean/N
#     return mean

In [3]:
# a = [0] * 10
# a = np.array(a)
# X = np.arange(8)
# print((a.T).shape)

In [4]:
def X(Xtrain, mean):
    Xdef = []
    for col in Xtrain.T:
        Xdef.append(col - mean)
    Xdef = np.array(Xdef)
    return Xdef.T

In [71]:
def covMat(X):
    C1 = np.matmul(X, X.T)
#     C2 = np.matmul(X.T, X)
    return C1

In [70]:
def eigenVectors(X):
    u, s, vh = linalg.svd(X, full_matrices=True)
    print('u: ', u)
    print('s: ', s)
    print('vh: ', vh)
    return u
def eigenVectors1(C):
    eigVal2, eigVect2 = linalg.eig(C)
    return eigVal2
#     eigVal2, eigVect2 = linalg.eig(C)
#     print("EigVect2: ", eigVect2.shape)
#     eigVect1 = np.matmul(X, eigVect2)
#     print("EigVect1: ", eigVect1.shape)
#     for i in range(0, ((eigVect1.T).shape)[0]):
#         (eigVect1.T)[i] = ((eigVect1.T)[i])/eigVal2[i]

In [7]:
def eigenWeights(E, x):
    weights = (E.T).dot(x)
    return weights

In [8]:
def face_detection(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cascade_image = cv2.CascadeClassifier('FaceDetHaarXML/haarcascade_frontalface_alt.xml')
    faces = cascade_image.detectMultiScale(image, scaleFactor = 1.3, minNeighbors = 5)
    if (len(faces) == 0):
        return None, None
    (x, y, l, b) = faces[0]
    return image[y:y+b, x:x+l], faces[0]

In [9]:
def loadDataLabels(data_path):
    faces = []
    labels = []
    enc_labels = []
    directories = os.listdir(data_path)
    for i, direc in enumerate(directories):
        sub_dir_path = data_path + "/" + direc
        sub_dir_images = os.listdir(sub_dir_path)
        label = direc
        for j, image_name in enumerate(sub_dir_images):
            image_path = sub_dir_path + "/" + image_name
            img = cv2.imread(image_path)
#             image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            face_detected, shape = face_detection(img)
            if face_detected is not None:
                face_detected = cv2.resize(face_detected, (256, 256))
                faces.append(face_detected)
                labels.append(label)
#                 enc_labels.appen(c)
#             if image is not None:
#                 image = cv2.resize(image, (256, 256))
#                 faces.append(image)
#                 labels.append(label)
    labels = np.array(labels)
    label_encoder = preprocessing.LabelEncoder()
    enc_labels = label_encoder.fit_transform(labels) 
    labels_unique = label_encoder.classes_
    faces = np.array(faces)
    labels_unique = np.array(labels_unique)
    enc_labels = np.array(enc_labels)
    return faces, labels, labels_unique, enc_labels

In [77]:
def weightMatrix(X_data):
    print('X_data: ', X_data)
    print ('X_data shape: ', X_data.shape)
    mean_ = mean(X_data)
    print('mean: ', mean_)
    print ('mean shape: ', mean_.shape)
    X_ = X(X_data, mean_)
    print('X: ', X_)
    print ('X shape: ', X_.shape)
#     C = covMat(X_)
#     print('C: ', C)
#     print ('C shape: ', C.shape)
    E = eigenVectors(X_)
#     E = eigenVectors1(C)
    print('E: ', E)
    print ('E shape: ', E.shape)
#     weightMatrix = eigenWeights(E, (X_data.T)[0])
#     for i in range(1, (X_data.T).shape[0]):
# #     for x_col in (X_data.T):
#         weightMatrix = np.vstack((weightMatrix, eigenWeights(E, ((X_data.T)[i]))))
    weightMatrix = np.matmul(E.T, X_data)
    print('wtMat: ', weightMatrix)
    print ('wtMat shape: ', weightMatrix.shape)
    return weightMatrix

In [63]:
def flattenData(X):
    X_data = []
    for image in X:
        X_data.append(image.flatten())
    X_data = np.array(X_data)
    return X_data.T

In [58]:
def PCAfamilyCoeff(X_train):
    X_data = flattenData(X_train)
    eigenCoeff = weightMatrix(X_data)
    return eigenCoeff

In [12]:
def drawEigenFaces(eigenCoeff, index):
    eigenFace = ((eigenCoeff.T)[index]).reshape(256, 256)
    plt.imshow(eigenFace)
    plt.show()

In [13]:
def euclideanDistance(vector1, vector2):
    return np.linalg.norm(vector1 - vector2)

In [14]:
def leastEucDist(matrix, vector):
    max_ind = -1
    max_ed = -1
    for i, col in range(((matrix.T).shape)[0]):
        euc_dist = euclideanDistance(vector, col) 
        if(max_ed < euc_dist):
            max_ed = euc_dist
            max_ind = i
    return max_ind

In [15]:
def PCA_prediction(X_train, y_train, X_test):
    train_eigenCoeff = PCAfamilyCoeff(X_train)
    test_eigenCoeff = PCAfamilyCoeff(X_test)
    predictions = []
    for test_col in test_eigenCoeff.T:
        predictions.append(leastEucDist(train_eigenCoeff, test_col))
    return predictions

In [16]:
def PCA_accuracy(predictions, y_test):
    count = 0
    if (len(predictions) != len(y_test)):
        print("Error...Mismatch in Lenghts!")
    for i in range(len(predictions)):
        if (predictions[i] == y_test[i]):
            count += 1
    print("PCA implementation Accuracy: ", (count/len(predictions))*100)

In [17]:
def Example(index, predictions, y_test, unique_labels):
    plt.imshow(X_test[index])
    plt.show()
    print("Prediction made by Model: ", unique_labels[predictions[index]])
    print("Ground Truth: ", unique_labels[y_test])

In [18]:
faces, labels, unique_labels, enc_labels = loadDataLabels("./Dataset1")
(X_train, X_test, y_train, y_test) = train_test_split(faces, enc_labels, test_size = 0.25, random_state = 42)

In [69]:
eigenCoeff = PCAfamilyCoeff(X_train)
# drawEigenFaces(eigenCoeff, 100)
# Example(100, predictions, y_test, unique_labels)

X_data:  [[ 57 116  60 ...   9 114 188]
 [ 55 140  51 ...  10 111 172]
 [ 57 136  60 ...   8 117 172]
 ...
 [105 223  18 ...   7 201 179]
 [102 223  29 ...   6 201 180]
 [105 224  23 ...  10 201 187]]
X_data shape:  (65536, 33)
mean:  [100.63636364  99.57575758  97.12121212 ...  78.75757576  78.84848485
  78.96969697]
mean shape:  (65536,)
X:  [[-43.63636364  15.36363636 -40.63636364 ... -91.63636364  13.36363636
   87.36363636]
 [-44.57575758  40.42424242 -48.57575758 ... -89.57575758  11.42424242
   72.42424242]
 [-40.12121212  38.87878788 -37.12121212 ... -89.12121212  19.87878788
   74.87878788]
 ...
 [ 26.24242424 144.24242424 -60.75757576 ... -71.75757576 122.24242424
  100.24242424]
 [ 23.15151515 144.15151515 -49.84848485 ... -72.84848485 122.15151515
  101.15151515]
 [ 26.03030303 145.03030303 -55.96969697 ... -68.96969697 122.03030303
  108.03030303]]
X shape:  (65536, 33)


MemoryError: 

In [ ]:
predictions = PCA_prediction(X_train, y_train, X_test)
PCA_accuracy(predictions, y_test)

In [ ]:
mean_ = mean(X_train)
# X(X_train, mean_)
print(mean_)

In [47]:
a = np.random.randn(9, 6) + 1j*np.random.randn(9, 6)
print(a.shape)
u, s, vh = np.linalg.svd(a, full_matrices=True)
print(X_train[30])    

(9, 6)
[[ 9 10  8 ...  2  3  5]
 [ 9  8  9 ...  4  3  3]
 [ 8  9  7 ...  4  2  2]
 ...
 [ 3  2  1 ...  7  7  6]
 [ 2  2  2 ...  9  9 10]
 [ 4  2  2 ...  7  6 10]]


In [55]:
PCAfamilyCoeff([X_train[0], X_train[30]])

[[ 57  55  57 ... 105 102 105]
 [  9  10   8 ...   7   6  10]]
X_data:  [[ 57   9]
 [ 55  10]
 [ 57   8]
 ...
 [105   7]
 [102   6]
 [105  10]]
X_data shape:  (65536, 2)
mean:  [33.  32.5 32.5 ... 56.  54.  57.5]
mean shape:  (65536,)
X:  [[ 24.  -24. ]
 [ 22.5 -22.5]
 [ 24.5 -24.5]
 ...
 [ 49.  -49. ]
 [ 48.  -48. ]
 [ 47.5 -47.5]]
X shape:  (65536, 2)
u:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
s:  [0. 1.]
vh:  [[1. 1.]
 [1. 0.]]
E:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
E shape:  (65536, 65536)
wtMat:  [[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]
wtMat shape:  (65536, 2)


array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [78]:
exd = flattenData([X_train[0], X_train[30], X_train[15]])
mean1 = mean(exd)
X1 = X(exd, mean1)
print(X1.shape)
print(X1)
# C1 = covMat(X1)
# print(C1)

(65536, 3)
[[-27.         -75.         102.        ]
 [-24.66666667 -69.66666667  94.33333333]
 [-20.         -69.          89.        ]
 ...
 [ 66.33333333 -31.66666667 -34.66666667]
 [ 64.66666667 -31.33333333 -33.33333333]
 [ 64.33333333 -30.66666667 -33.66666667]]
